In [94]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

!pip install geopy
from geopy.distance import geodesic

In [95]:
# Import datasets from csv files
sample = pd.read_csv('Sample_Submission.csv')

delivery = pd.read_csv('train.csv')

In [96]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11399 entries, 0 to 11398
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                11399 non-null  object 
 1   Time_taken (min)  11399 non-null  float64
dtypes: float64(1), object(1)
memory usage: 178.2+ KB


In [97]:
sample.head()

,ID,Time_taken (min)
0,0x2318,25.668333
1,0x3474,27.881667
2,0x9420,27.023333
3,0x72ee,28.153333
4,0xa759,21.018333


In [98]:
delivery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

In [99]:
delivery.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [100]:
# Check for missing values
missing = pd.concat([(delivery.isnull().sum()), 100 * delivery.isnull().mean()], axis=1, keys=['count', '%'])
missing.columns = ('count', '%')
missing.sort_values(by=['count', '%'], ascending=[True, False])

,count,%
ID,0,0.0
Delivery_person_ID,0,0.0
Delivery_person_Age,0,0.0
Delivery_person_Ratings,0,0.0
Restaurant_latitude,0,0.0
Restaurant_longitude,0,0.0
Delivery_location_latitude,0,0.0
Delivery_location_longitude,0,0.0
Order_Date,0,0.0
Time_Orderd,0,0.0


In [101]:
# Check for any duplicate ID values
delivery['ID'].value_counts().head()

0x4607     1
0x1f3e     1
0xe251     1
0x3f31     1
0x4a78     1
Name: ID, dtype: int64

In [102]:
# Delete Vehicle_condition column: unclear values
del delivery['Vehicle_condition']
print(delivery.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Type_of_order   

Column descriptions:

- ID: unique identifier of the individual delivery orders. 
- Delivery_person_ID: unique identifier of the person completing the delivery route. 
- Delivery_person_Age: age of the person completing the delivery route. 
- Delivery_person_Ratings: rating between 1 and 5 (1=worst, 5=best) of the person completing the delivery route. 

- Restaurant_latitude: north-south coordinate of the restaurant location as the delivery starting point. 
- Restaurant_longitude: east-west coordinate of the restaurant location as the delivery starting point. 
- Delivery_latitude: north-south coordinate of the customer location as the delivery end point. 
- Delivery_longitude: east-west coordinate of the customer location as the delivery end point. 

- Order_Date: the date that the order was placed on in dd-mm-yyyy format. 
- Time_Orderd: the time of day that the order was placed. 
- Time_Order_Picked: the time of day that the order was picked up from restaurant supplier. 

- Weatherconditions: the state of the weather at the time the order was placed. 
- Road_traffic_density: how dense the traffic conditions were during delivery period. 
- Vehicle_condition: unclear int value, does not seem applicable -- column deleted. 
- Type_of_order: 

In [103]:
# Tidy the Weatherconditions column by removing redundant 'conditions' string 
# Tidy the Time_taken(min) column by removing redundant '(min)' string

delivery['Weatherconditions'] = delivery['Weatherconditions'].str.replace('conditions', '')
delivery['Time_taken(min)'] = delivery['Time_taken(min)'].str.replace('[(min)]', '')
delivery.head()

/var/folders/x1/k6x2x2c9677_n0slz_pq1p5r0000gn/T/ipykernel_68941/4191341627.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  delivery['Time_taken(min)'] = delivery['Time_taken(min)'].str.replace('[(min)]', '')


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,Sunny,High,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,Stormy,Jam,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,Sandstorms,Low,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,Sunny,Medium,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,Cloudy,High,Snack,scooter,1,No,Metropolitian,30


In [115]:
# Tidy the multiple_deliveries column by observing NaN values
delivery['multiple_deliveries'].unique()

array(['0', '1', '3', 'NaN ', '2'], dtype=object)

In [134]:
int(delivery['multiple_deliveries'])

TypeError: cannot convert the series to <class 'int'>

In [135]:
# Remove white spaces after 'NaN '
mult = delivery['multiple_deliveries']
mult = list(map(lambda x: x.replace('NaN ', ''), mult))


In [126]:
delivery['multiple_deliveries'].value_counts().head()

1       28159
0       14095
2        1985
NaN       993
3         361
Name: multiple_deliveries, dtype: int64

In [130]:
delivery.shape

(45593, 19)

In [127]:
'''Coming back to this, need to figure out where NaN values are located and how to replace them.'''

'Coming back to this, need to figure out where NaN values are located and how to replace them.'